# Sommaire
- Importation et aperçu du fichier CSV
    - Importation du CSV
    - Visualisation du CSV
- Traitement sur les colonnes
    - Sélection des colonnes utiles
    - Transformation des colonnes sélectionnées
- Détection et suppression des valeurs aberrantes
    - Variables numériques
    - Variables catégoriques 
- Vérification et exportation du DataFrame
    - DataFrame finalisé
    - Exportation du DataFrame

_ _ _ 

# Importation et aperçu du fichier CSV

## Importation du CSV

Importation des modules nécessaires :

In [58]:
# Importation des modules nécessaires
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, count
from pyspark.sql.types import FloatType

Création d'une SparkSession sur la machine local avec 3 threads dédiés :

In [59]:
# Création d'une SparkSession
spark = SparkSession.builder \
    .appName("Open Food Facts") \
    .master("local[3]") \
    .getOrCreate()

Récupération du csv OpenFoodFacts :

In [60]:
# Récupération du csv OpenFoodFacts
df = spark.read.csv("./Projet_OpenFoodFacts/en.openfoodfacts.org.products3.csv", header=True, sep="\t")

## Visualisation du CSV

Aperçu de la structure des données ainsi que du noms des colonnes :

In [61]:
# Aperçu de la structure des données ainsi que du noms des colonnes
df.printSchema()

root
 |-- code: string (nullable = true)
 |-- url: string (nullable = true)
 |-- creator: string (nullable = true)
 |-- created_t: string (nullable = true)
 |-- created_datetime: string (nullable = true)
 |-- last_modified_t: string (nullable = true)
 |-- last_modified_datetime: string (nullable = true)
 |-- last_modified_by: string (nullable = true)
 |-- last_updated_t: string (nullable = true)
 |-- last_updated_datetime: string (nullable = true)
 |-- product_name: string (nullable = true)
 |-- abbreviated_product_name: string (nullable = true)
 |-- generic_name: string (nullable = true)
 |-- quantity: string (nullable = true)
 |-- packaging: string (nullable = true)
 |-- packaging_tags: string (nullable = true)
 |-- packaging_en: string (nullable = true)
 |-- packaging_text: string (nullable = true)
 |-- brands: string (nullable = true)
 |-- brands_tags: string (nullable = true)
 |-- categories: string (nullable = true)
 |-- categories_tags: string (nullable = true)
 |-- categories_e

Connaître le nombre de colonnes:

In [62]:
# Obtenir le nombre de colonnes

nb_colonnes = len(df.columns)

print(f"\nNombre de colonnes : {nb_colonnes}\n")


Nombre de colonnes : 206



Nous avons donc 206 colonnes à examiner afin de savoir celles que l'on garde et celles que l'on supprime.

Connaître le nombre de lignes :

In [63]:
# Nombre de lignes
print(df.count())

3083356


Affichage des 10 premières lignes :

In [64]:
# Affichage des 10 premières lignes
df.show(10)

+--------------------+--------------------+--------------+----------+--------------------+---------------+----------------------+----------------+--------------+---------------------+--------------------+------------------------+------------+--------+---------+--------------+------------+--------------+-------+-----------+--------------------+--------------------+--------------------+-------+------------+----------+--------------------+-------------------------+------------+------------+---------+---------+--------------+------------------------+------+-----------+---------------+------+---------+----------------+-------------+--------------------+--------------------+-------------------------+---------+------------+------+-----------+---------+------------+----------------+-----------------+-----------+---------+--------------+------------+----------------+----------------+----------+--------------------+--------------+-----------------+--------------------+--------------------+------

_ _ _

# Traitement sur les colonnes

## Sélection des colonnes utiles

Suppression des colonnes entièrement vides

In [65]:
# Suppression des colonnes entièrement vides :

# Calculer le nombre de valeurs non NULL pour chaque colonne
non_null_counts = df.agg(*(count(c).alias(c) for c in df.columns))

# Collecter les statistiques et filtrer les colonnes avec uniquement des valeurs NULL
non_null_counts_collected = non_null_counts.collect()[0].asDict()
columns_to_drop = [c for c, count in non_null_counts_collected.items() if count == 0]

# Supprimer les colonnes avec uniquement des valeurs NULL
df_cleaned = df.drop(*columns_to_drop)

# Afficher le schéma et les premières lignes du DataFrame nettoyé pour vérifier
#df_cleaned.printSchema()
#df_cleaned.show(10)
print(f"\nColonnes à supprimer : {columns_to_drop}")


Colonnes à supprimer : ['cities', 'allergens_en', 'additives']


Création d'une vue temporaire du fichier csv afin de vérifier nos colonnes avant validation ou suppression

Les colonnes sont examinées une à une en fonction de : 
- l'occurrence de leurs valeurs
- la pertinence des valeurs remontées

In [66]:
# Création d'une vue temporaire
df_cleaned.createOrReplaceTempView("open_food_facts")

# Requête SQL sélectionnant les lignes non vides d'une colonne spécifique
empty_rows = spark.sql("""
SELECT COUNT(acidity_100g)
FROM open_food_facts
WHERE acidity_100g IS NOT NULL;
""")

# Afficher le résultat de la requête
empty_rows.show()


# Requête SQL sélectionnant les lignes non vides d'une colonne spécifique
rows_values = spark.sql("""
SELECT DISTINCT acidity_100g
FROM open_food_facts
WHERE acidity_100g IS NOT NULL
LIMIT 15;
""")

# Afficher le résultat de la requête sans troncation
rows_values.show(truncate=False)


+-------------------+
|count(acidity_100g)|
+-------------------+
|                  7|
+-------------------+

+------------+
|acidity_100g|
+------------+
|6           |
|0.39        |
|5           |
|8           |
|4           |
+------------+



Voici le détail de la sélection de nos colonnes :

Les colonnes doivent être utiles pour notre devoir et posséder un nombre d'occurrences suffisants pour être gardées.<br>
Les colonnes inutiles, dont les informations sont équivoques et dont les occurrences ne sont pas suffisantes ne seront pas conservées.

- code ==> Inutile
- url ==> Inutile
- creator ==> Inutile
- created_t ==> Inutile
- created_datetime ==> Inutile
- last_modified_t ==> Inutie
- last_modified_datetime ==> Inutile
- last_modified_by ==> Inutie
- last_updated_t ==> Inutile
- last_updated_datetime ==> Inutile
- product_name ==> A Garder (name)
- abbreviated_product_name ==> Utile mais la colonne product_name convient mieux
- generic_name ==> Utile mais la colonne product_name convient mieux
- quantity ==> Inutile
- packaging ==> Inutie
- packaging_tags ==> Inutile
- packaging_en ==> Inutile
- packaging_text ==> Inutile
- brands ==> Inutile
- brands_tags ==> Inutile
- categories ==> Trop général, la colonne food_groups_tags convient mieux pour déterminer le groupe alimentaire auquel appartient chaque aliment
- categories_tags ==> Trop général, la colonne food_groups_tags convient mieux pour déterminer le groupe alimentaire auquel appartient chaque aliment
- categories_en ==> Trop général, la colonne food_groups_tags convient mieux pour déterminer le groupe alimentaire auquel appartient chaque aliment
- origins ==> Utile mais avec seulement 127259 occurrences, la colonne countries_tags convient mieux
- origins_tags ==> Utile mais avec seulement 127124 occurrences, la colonne countries_tags convient mieux
- origins_en ==> Utile mais avec seulement 127124 occurences, la colonne countries_tags convient mieux
- manufacturing_places ==> Inutile
- manufacturing_places_tags ==> Inutile
- labels ==> Utile, occurrences suffisantes, mais trop équivoque et complexe à traiter
- labels_tags ==> Utile, occurrences suffisantes, mais trop équivoque et complexe à traiter
- labels_en ==> Utile, occurrences suffisantes, mais trop équivoque et complexe à traiter
- emb_codes ==> Inutile
- emb_codes_tags ==> Inutile
- first_packaging_code_geo ==> Inutile
- cities ==> Colonne vide
- cities_tags ==> Inutile
- purchase_places ==> Intéressant, mais la colonne countries_tags convient mieux
- stores ==> Inutile
- countries ==> La colonne countries_tags convient mieux
- countries_tags ==> A Garder (localisation),car les enegistrements sont au même format, contrairement à countries et countries_en
- countries_en ==> La colonne countries_tags convient mieux
- ingredients_text ==> Trop complexe à exploiter
- ingredients_tags ==> Trop complexe à exploiter
- ingredients_analysis_tags ==> Trop complexe à exploiter
- allergens ==> Utile mais pas assez d'occurrences, seulement 262069
- allergens_en ==> Colonne vide
- traces ==> Utile, pour déterminer les résidus potentiellement allergène  mais pas assez d'occurences, seulement 136204
- traces_tags ==> Utile, pour déterminer les résidus potentiellement allergène  mais pas assez d'occurences
- traces_en ==> Utile, pour déterminer les résidus potentiellement allergène  mais pas assez d'occurences
- serving_size ==> Inutile
- serving_quantity ==> Inutile
- no_nutrition_data ==> Inutile
- additives_n ==> Inutile
- additives ==> Colonne vide
- additives_tags ==> Inutile
- additives_en ==> Inutile
- nutriscore_score ==> Inutile
- nutriscore_grade ==> Inutile
- nova_group ==> A Garder car intéressant pour la santé du consommateur en bannissant les aliments du groupe 4 qui favorisent le développement de cancers
- pnns_groups_1 ==> Intéressant, mais de niveau national et non international
- pnns_groups_2 ==> Intéressant, mais de niveau national et non international
- food_groups ==> Intéressant, mais trop complexe à traiter
- food_groups_tags ==> A Garder (catégorie), nous donne un groupe par aliment, cette colonne sera utile pour nos tris futurs
- food_groups_en ==> Intéressant mais trop complexe à traiter
- states ==> Données à compléter ...
- states_tags ==> Données à compléter ...
- states_en ==> Données à compléter ...
- brand_owner ==> Inutile
- ecoscore_score ==> Intéressant mais trop complexe à analyser, il nous faut grouper ces chiffres afin de les rendre plus lisibles : la colonne ecoscore_grade convient donc mieux
- ecoscore_grade ==> A Garder, car tournée vers une alimentation plus durable et qui se soucie de notre santé. Seuls les aliments dont les ecoscores notés A et B seront conservés
- nutrient_levels_tags ==> Intéressant mais pas suffisamment précis pour la sélection de nos aliments
- product_quantity ==> Inutile
- owner ==> Inutile
- data_quality_errors_tags ==> Inutile
- unique_scans_n ==> Inutile
- popularity_tags ==> Inutile
- completeness ==> Inutile
- last_image_t ==> Inutile
- last_image_datetime ==> Inutile
- main_category ==> Intéressant mais les catégories sont trop spécifiques, nous préférons la colonne food_groups_tags
- main_category_en ==> Intéressant mais les catégories sont trop spécifiques, nous préférons la colonne food_groups_tags
- image_url ==> Inutile
- image_small_url ==> Inutile
- image_ingredients_url ==> Inutile
- image_ingredients_small_url ==> Inutile
- image_nutrition_url ==> Inutile
- image_nutrition_small_url ==> Inutile
- energy-kj_100g ==> Intéressant mais l'énergie en joules est moins répandue que l'énergie en calories lorsque cela concerne la nourriture, nous privilégeons donc la colonne energy-kcal_100g
- energy-kcal_100g ==> A Garder (calories), nous avons la quantité de calories pour 100 grammes  
- energy_100g ==> Pas d'unité ... Nous privilégeons donc la colonne energy-kcal_100g
- energy-from-fat_100g ==> Inutile
- fat_100g ==> Trop général, il nous faut davantage de précision, seuls les gras insaturés, bons pour la santé sont à privilégier
- saturated-fat_100g ==> A garder afin d'effectuer un tri et supprimer les aliments en contenant une certaine quantité
- butyric-acid_100g ==> Pas assez d'occurrences
- caproic-acid_100g ==> Pas assez d'occurrences
- caprylic-acid_100g ==> Pas assez d'occurrences
- capric-acid_100g ==> Pas assez d'occurrences
- lauric-acid_100g ==> Pas assez d'occurrences
- myristic-acid_100g ==> Pas assez d'occurrences
- palmitic-acid_100g ==> Pas assez d'occurrences
- stearic-acid_100g ==> Pas assez d'occurrences
- arachidic-acid_100g ==> Pas assez d'occurrences
- behenic-acid_100g ==> Pas assez d'occurrences
- lignoceric-acid_100g ==> Pas assez d'occurrences
- cerotic-acid_100g ==> Pas assez d'occurrences
- montanic-acid_100g ==> Pas assez d'occurrences
- melissic-acid_100g ==> Pas assez d'occurrences
- unsaturated-fat_100g ==> Pas assez d'occurrences
- monounsaturated-fat_100g ==> Pas assez d'occurences
- omega-9-fat_100g ==> Pas assez d'occurrences
- polyunsaturated-fat_100g ==> Pas assez d'occurrences
- omega-3-fat_100g ==> Pas assez d'occurrences
- omega-6-fat_100g ==> Pas assez d'occurrences
- alpha-linolenic-acid_100g ==>Pas assez d'occurrences
- eicosapentaenoic-acid_100g ==> Pas assez d'occurrences
- docosahexaenoic-acid_100g ==> Pas assez d'occurrences
- linoleic-acid_100g ==> Pas assez d'occurrences
- arachidonic-acid_100g ==> Pas assez d'occurrences
- gamma-linolenic-acid_100g ==> Pas assez d'occurrences
- dihomo-gamma-linolenic-acid_100g ==> Pas assez d'occurrences
- oleic-acid_100g ==> Pas assez d'occurrences 
- elaidic-acid_100g ==> Pas assez d'occurrences
- gondoic-acid_100g ==> Pas assez d'occurrences
- mead-acid_100g ==> Pas assez d'occurrences
- erucic-acid_100g ==> Pas assez d'occurrences
- nervonic-acid_100g ==> Pas assez d'occurrences
- trans-fat_100g ==> Pas assez d'occurences (environ 270 000)
- cholesterol_100g ==> Pas assez d'occurences (environ 280 000)
- carbohydrates_100g ==> A Garder (carbohydrates)
- sugars_100g ==> A Garder (sugars)
- added-sugars_100g ==> Pas assez d'occurrences
- sucrose_100g ==> Pas assez d'occurrences
- glucose_100g ==> Pas assez d'occurrences
- fructose_100g ==> Pas assez d'occurrences
- lactose_100g ==> Pas assez d'occurrences
- maltose_100g ==> Pas assez d'occurerences
- maltodextrins_100g ==> Pas assez d'occurerences
- starch_100g ==> Pas assez d'occurerences
- polyols_100g ==> Pas assez d'occurerences
- erythritol_100g ==> Pas assez d'occurrences
- fiber_100g ==> A Garder (fiber)
- soluble-fiber_100g ==> Pas assez d'occurrences
- insoluble-fiber_100g ==> Pas assez d'occurrences
- proteins_100g ==> A Garder (proteins)
- casein_100g ==> Pas assez d'occurrences
- serum-proteins_100g ==> Pas assez d'occurrences
- nucleotides_100g ==> Pas assez d'occurrences
- salt_100g ==> A Garder (salt)
- added-salt_100g ==> Pas assez d'occurrences
- sodium_100g ==> A Garder (sodium)
- alcohol_100g ==> Pas assez d'occurrences
- vitamin-a_100g ==> Pas assez d'occurrences
- beta-carotene_100g ==> Pas assez d'occurrences
- vitamin-d_100g ==> Pas assez d'occurrences
- vitamin-e_100g ==> Pas assez d'occurrences
- vitamin-k_100g ==> Pas assez d'occurrences
- vitamin-c_100g ==> A Garder (vitamin-c)
- vitamin-b1_100g ==> Pas assez d'occurrences
- vitamin-b2_100g ==> Pas assez d'occurrences
- vitamin-pp_100g ==> Pas assez d'occurrences
- vitamin-b6_100g ==> Pas assez d'occurrences
- vitamin-b9_100g ==> Pas assez d'occurrences
- folates_100g ==> Pas assez d'occurrences
- vitamin-b12_100g ==> Pas assez d'occurrences
- biotin_100g ==> Pas assez d'occurrences
- pantothenic-acid_100g ==> Pas assez d'occurrences
- silica_100g ==> Pas assez d'occurrences
- bicarbonate_100g ==> Pas assez d'occurrences
- potassium_100g ==> Pas assez d'occurrences
- chloride_100g ==> Pas assez d'occurrences 
- calcium_100g ==> Pas assez d'occurrences, environ 280 000
- phosphorus_100g ==> Pas assez d'occurrences
- iron_100g ==> Pas assez d'occurrences, environ 255 000
- magnesium_100g ==> Pas assez d'occurrences
- zinc_100g ==> Pas assez d'occurrences
- copper_100g ==> Pas assez d'occurrences
- manganese_100g ==> Pas assez d'occurrences
- fluoride_100g ==> Pas assez d'occurrences
- selenium_100g ==> Pas assez d'occurrences
- chromium_100g ==> Pas assez d'occurrences
- molybdenum_100g ==> Pas assez d'occurrences 
- iodine_100g ==> Pas assez d'occurrences 
- caffeine_100g ==> Pas assez d'occurrences
- taurine_100g ==> Pas assez d'occurrences
- ph_100g ==> Pas assez d'occurrences
- fruits-vegetables-nuts_100g ==> Pas assez d'occurrences
- fruits-vegetables-nuts-dried_100g ==> Pas assez d'occurrences
- fruits-vegetables-nuts-estimate_100g ==> Pas assez d'occurrences
- fruits-vegetables-nuts-estimate-from-ingredients_100g ==> Presqu'assez d'occurrences
- collagen-meat-protein-ratio_100g ==> Pas assez d'occurrences
- cocoa_100g ==> Pas assez d'occurrences
- chlorophyl_100g ==> Pas assez d'occurrences
- carbon-footprint_100g ==> Pas assez d'occurrences
- carbon-footprint-from-meat-or-fish_100g ==> Pas assez d'occurrences 
- nutrition-score-fr_100g ==> Inutile
- nutrition-score-uk_100g ==> Inutile
- glycemic-index_100g ==> Pas assez d'occurrences
- water-hardness_100g ==> Inutile
- choline_100g ==> Pas assez d'occurrences
- phylloquinone_100g ==> Pas assez d'occurrences
- beta-glucan_100g ==> Pas assez d'occurrences
- inositol_100g ==> Pas assez d'occurrences
- carnitine_100g ==> Pas assez d'occurrences
- sulphate_100g ==> Pas assez d'occurrences
- nitrate_100g ==> Pas assez d'occurrences
- acidity_100g ==> Pas assez d'occurrences


Ainsi nous travaillerons sur les colonnes suivantes :

- product_name (name)
- countries_tags (localisation)
- nova_group ==> nous la conserverons temporairement car elle est intéressante pour la santé du consommateur. Nous bannirons les aliments du groupe 4 qui favorisent le développement de cancers.
- food_groups_tags ==> (categorie), nous donne un groupe par aliment, cette colonne sera utile pour nos tris futurs.
- ecoscore_grade ==> nous la conserverons temporairement car elle est tournée vers une alimentation plus durable et qui se soucie de la santé du consommateur. Seuls les aliments dont les ecoscores notés A et B seront conservés.
- energy-kcal_100g ==> (calories), nous avons la quantité de calories pour 100 grammes.
- saturated-fat_100g ==> nous la conserverons temporairement afin d'effectuer un tri et supprimer les aliments en contenant une certaine quantité.
- carbohydrates_100g ==> (carbohydrates)
- sugars_100g ==> (sugars)
- fiber_100g ==> (fiber)
- proteins_100g ==> (proteins)
- salt_100g ==> (salt)
- sodium_100g ==> (sodium)
- vitamin-c_100g ==> (vitamin-c)






Suppression des colonnes inutiles :

In [67]:
# Nous entrons dans une variable les colonnes à garder
colonnes_a_garder = ["product_name","countries_tags", "nova_group", "food_groups_tags", "ecoscore_grade", "energy-kcal_100g", "saturated-fat_100g", "carbohydrates_100g", "sugars_100g", "fiber_100g", "proteins_100g", "salt_100g", "sodium_100g", "vitamin-c_100g"]

# Nous sélectionnons uniquement les colonnes à conserver
df_cleaned_2 = df_cleaned.select(*colonnes_a_garder)

# Nous affichons notre nouveau DataFrame
df_cleaned_2.show(truncate=False)

+-----------------------------+----------------+----------+--------------------------------------------+--------------+----------------+------------------+------------------+-----------+----------+-------------+---------+-----------+--------------+
|product_name                 |countries_tags  |nova_group|food_groups_tags                            |ecoscore_grade|energy-kcal_100g|saturated-fat_100g|carbohydrates_100g|sugars_100g|fiber_100g|proteins_100g|salt_100g|sodium_100g|vitamin-c_100g|
+-----------------------------+----------------+----------+--------------------------------------------+--------------+----------------+------------------+------------------+-----------+----------+-------------+---------+-----------+--------------+
|jeunes pousses               |en:france       |NULL      |en:fruits-and-vegetables,en:vegetables      |b             |NULL            |NULL              |NULL              |NULL       |NULL      |NULL         |NULL     |NULL       |NULL          |
|And

Nous devons ensuite procéder à la suppression de toutes les lignes possédant au moins une valeur de type NULL :

In [68]:
# Suppression des lignes avec des valeurs NULL
df_cleaned_3 = df_cleaned_2.dropna()

# Nous affichons notre nouveau DataFrame
df_cleaned_3.show(truncate=False)

# Affichons le nombre de lignes de notre nouveau DataFrame
print(f"Nombre de lignes : {df_cleaned_3.count()}")

+-----------------------------------------+----------------+----------+--------------------------------------------+--------------+----------------+------------------+------------------+-----------+----------+-------------+---------+-----------+--------------+
|product_name                             |countries_tags  |nova_group|food_groups_tags                            |ecoscore_grade|energy-kcal_100g|saturated-fat_100g|carbohydrates_100g|sugars_100g|fiber_100g|proteins_100g|salt_100g|sodium_100g|vitamin-c_100g|
+-----------------------------------------+----------------+----------+--------------------------------------------+--------------+----------------+------------------+------------------+-----------+----------+-------------+---------+-----------+--------------+
|Cranberries                              |en:united-states|3         |en:fruits-and-vegetables,en:fruits          |d             |300             |0                 |83.33             |66.67      |10        |0       

Nombre de lignes : 137316


## Transformation des colonnes sélectionnées

Nous supprimons maintenant toutes les lignes dont les valeurs de la colonne nova_group sont égales à 4.</br>
Ces aliments peuvent potentiellement développer des cancers.</br>
Nous supprimons ensuite cette colonne.


In [69]:
# Nous filtrons les lignes dont la valeur de la colonne "nova_group" vaut 1, 2 ou 3
df_cleaned_4 = df_cleaned_3.filter(df_cleaned_3["nova_group"] .isin([1, 2, 3]))

# Nous supprimons ensuite la colonne "nova_group"
df_cleaned_5 = df_cleaned_4.drop("nova_group")

# Nous affichons notre nouveau DataFrame
df_cleaned_5.show(truncate=False)

# Affichons le nombre de lignes de notre nouveau DataFrame
print(f"Nombre de lignes : {df_cleaned_5.count()}")

+------------------------------------------------------------+----------------+------------------------------------------+--------------+----------------+------------------+------------------+-----------+----------+-------------+---------+-----------+--------------+
|product_name                                                |countries_tags  |food_groups_tags                          |ecoscore_grade|energy-kcal_100g|saturated-fat_100g|carbohydrates_100g|sugars_100g|fiber_100g|proteins_100g|salt_100g|sodium_100g|vitamin-c_100g|
+------------------------------------------------------------+----------------+------------------------------------------+--------------+----------------+------------------+------------------+-----------+----------+-------------+---------+-----------+--------------+
|Cranberries                                                 |en:united-states|en:fruits-and-vegetables,en:fruits        |d             |300             |0                 |83.33             |66.67  

Nombre de lignes : 46677


Nous supprimons maintenant toutes les lignes dont les valeurs de la colonne ecoscore_grade valent autre chose que A ou B.</br>
L'impact environnemental de ces aliments est trop nocif.</br>
Nous supprimons ensuite cette colonne.

In [70]:
# Nous filtrons les lignes dont la valeur de la colonne "ecoscore_grade" vaut A ou B
df_cleaned_6 = df_cleaned_5.filter(df_cleaned_5["ecoscore_grade"] .isin(["a", "b"]))

# Nous supprimons ensuite la colonne "ecoscore_grade"
df_cleaned_7 = df_cleaned_6.drop("ecoscore_grade")

# Nous affichons notre nouveau DataFrame
df_cleaned_7.show(truncate=False)

# Affichons le nombre de lignes de notre nouveau DataFrame
print(f"Nombre de lignes : {df_cleaned_7.count()}")

+---------------------------------------------+--------------------------+---------------------------------------------+----------------+------------------+------------------+-----------+----------+-------------+----------+-----------+--------------+
|product_name                                 |countries_tags            |food_groups_tags                             |energy-kcal_100g|saturated-fat_100g|carbohydrates_100g|sugars_100g|fiber_100g|proteins_100g|salt_100g |sodium_100g|vitamin-c_100g|
+---------------------------------------------+--------------------------+---------------------------------------------+----------------+------------------+------------------+-----------+----------+-------------+----------+-----------+--------------+
|Creamed Honey                                |en:united-states          |en:sugary-snacks,en:sweets                   |286             |0                 |80.95             |76.19      |0         |0            |0         |0          |0           

Nombre de lignes : 6596


Afin de ne garder que les ingrédients nécessaires à nos futurs régimes alimentaires, il nous faut lister les différentes catégories présentes dans la colonne food_groups_tags, puis retirer celles jugées inappropriées.

In [71]:
# Nous sélectionnons les différentes catégories
categories_distinctes = df_cleaned_7.select("food_groups_tags").distinct()

# Nous affichons ces catégories
categories_distinctes.show(100, truncate=False)

+----------------------------------------------------+
|food_groups_tags                                    |
+----------------------------------------------------+
|en:salty-snacks,en:appetizers                       |
|en:sugary-snacks,en:biscuits-and-cakes              |
|en:cereals-and-potatoes,en:bread                    |
|en:cereals-and-potatoes,en:legumes                  |
|en:composite-foods,en:one-dish-meals                |
|en:fruits-and-vegetables,en:vegetables              |
|en:fruits-and-vegetables,en:dried-fruits            |
|en:salty-snacks,en:salty-and-fatty-products         |
|en:fish-meat-eggs,en:meat,en:meat-other-than-poultry|
|en:fats-and-sauces,en:dressings-and-sauces          |
|en:beverages,en:unsweetened-beverages               |
|en:cereals-and-potatoes,en:potatoes                 |
|en:cereals-and-potatoes,en:cereals                  |
|en:fruits-and-vegetables                            |
|en:composite-foods,en:pizza-pies-and-quiches        |
|en:milk-a

Suite à ce résultat, nous supprimerons toutes les catégories contenant les mots :
- unknown
- fat
- sugar
- ice-cream
- cake
- pizza
- salt

Les aliments présents dans ces catégories ne sont en effet pas compatibles avec un régime vertueux.

In [72]:
# Nous construisons une expression régulières pour les mots à exclure
regex_pattern = ".*(unknown|fat|sugar|ice-cream|cake|pizza|salt).*"

# Sélection des catégories ne contenant pas les mots ci-dessus
df_cleaned_8 = df_cleaned_7.filter(~col("food_groups_tags").rlike(regex_pattern))

# Nous sélectionnons les valeurs distinctes de notre colonne food_groups_tags
categories_distinctes = df_cleaned_8.select("food_groups_tags").distinct()

# Nous affichons cette colonne
categories_distinctes.show(100, truncate=False)

# Affichons le nombre de lignes de notre nouveau DataFrame suite à ces suppressions
print(f"Nombre de lignes : {df_cleaned_8.count()}")

+----------------------------------------------------+
|food_groups_tags                                    |
+----------------------------------------------------+
|en:cereals-and-potatoes,en:bread                    |
|en:cereals-and-potatoes,en:legumes                  |
|en:composite-foods,en:one-dish-meals                |
|en:fruits-and-vegetables,en:vegetables              |
|en:fruits-and-vegetables,en:dried-fruits            |
|en:fish-meat-eggs,en:meat,en:meat-other-than-poultry|
|en:beverages,en:unsweetened-beverages               |
|en:cereals-and-potatoes,en:potatoes                 |
|en:cereals-and-potatoes,en:cereals                  |
|en:fruits-and-vegetables                            |
|en:milk-and-dairy-products,en:milk-and-yogurt       |
|en:cereals-and-potatoes,en:breakfast-cereals        |
|en:fruits-and-vegetables,en:fruits                  |
|en:beverages,en:sweetened-beverages                 |
|en:beverages,en:plant-based-milk-substitutes        |
|en:fruits

Nous supprimons maintenant les lignes entièrement dupliquées, c'est-à-dire où chaque valeur de colonne dans une ligne est une copie exacte d'une autre ligne, grâce à la fonction distinct(). Nous aurions également pu utiliser la fonction dropDuplicates().

In [73]:
df_cleaned_9 = df_cleaned_8.distinct()

print(f"Nombre de lignes : {df_cleaned_9.count()}")

Nombre de lignes : 4289


# Détection et suppression des valeurs aberrantes

Enfin, pour terminer, il ne nous reste plus qu'à sélectionner les valeurs aberrantes de chacunes de nos colonnes et identifier pour chacunes d'entre elles s'il s'agit d'erreurs expérimentales, auquel cas nous les supprimerons.<br>

## Variables numériques

Concernant nos variables de type numérique, nous procéderons à cette détection à l'aide d'une méthode statistique basée sur les écarts interquartiles ou IDR.<br> 
Les valeurs seront considérées comme aberrantes si elles se trouvent en dehors de l'intervalle [Q1 - 1.5 * IQR, Q3 + 1.5 * IQR].

Nous stockerons nos colonnes contenant des variables de type numérique dans une liste et nous bouclerons sur chacune de ces colonnes afin de garder dans notre DataFrame seulement les valeurs qui ne sont pas aberrantes.

In [74]:
# Liste des colonnes pour lesquelles détecter les valeurs aberrantes
colonnes_numériques = ['energy-kcal_100g', 'saturated-fat_100g', 'carbohydrates_100g', 'sugars_100g', 'fiber_100g', 'proteins_100g', 'salt_100g', 'sodium_100g', 'vitamin-c_100g']

# Convertir chaque colonne spécifiée en float
for colonne in colonnes_numériques:
    df_cleaned_9 = df_cleaned_9.withColumn(colonne, col(colonne).cast(FloatType()))

# Nous bouclons sur notre liste de colonnes afin de calculer notre plage de valeurs acceptables
for colonne in colonnes_numériques:
    # Calcul des quantiles pour la colonne courante
    quantiles = df_cleaned_9.approxQuantile(colonne, [0.25, 0.75], 0.05)  # La précision de l'estimation
    Q1, Q3 = quantiles
    IQR = Q3 - Q1
    borne_inf = Q1 - 1.5 * IQR
    borne_sup = Q3 + 1.5 * IQR
    
    # Nous enregistrons nos colonnes sans valeurs aberrantes dans notre DataFrame
    df_cleaned_10 = df_cleaned_9.filter((col(colonne) >= borne_inf) & (col(colonne) <= borne_sup))
    
# Nous affichons le nombre de ligne total de notre nouveau DataFrame
print(f"\nNombre total de lignes : {df_cleaned_9.count()}")


Nombre total de lignes : 4289



## Variables catégoriques

Concernant nos variables de type catégorique, nous procéderons à cette détection en utilisant une méthode basée sur l'analyse d'un type de format prédéfini à l'aide de regex. Ainsi, les catégories n'appartenant pas à ce type de format pré-établi, seront considérées comme des valeurs aberrantes, et nous devrons vérifier manuellement si elles repésentent ou non des erreurs expérimentales.

Nous effectuerons notre analyse sur les colonnes countries_tags et food_groups_tags.<br>
Nous considérons comme des valeurs aberrantes, les catégories dont le format contient au moins un chiffre.

Colonne countries_tags :

In [75]:
# Sélection des catégories contenant au moins un chiffre
aberr_countries_tags = df_cleaned_10.filter(col("countries_tags").rlike(".*[0-9].*")).select("countries_tags").distinct()

# Affichage de ces valeurs aberrantes
aberr_countries_tags.show(truncate=False)

+--------------+
|countries_tags|
+--------------+
+--------------+



En nous basant sur un format ne contenant aucuns chiffres, aucune valeur aberrante n'a été détectée pour notre colonne countries_tags.

Colonne food_groups_tags :

In [76]:
# Sélection des catégories contenant au moins un chiffre
aberr_food_groups_tags = df_cleaned_10.filter(col("food_groups_tags").rlike(".*[0-9].*")).select("food_groups_tags").distinct()

# Affichage de ces valeurs aberrantes
aberr_food_groups_tags.show(truncate=False)

+----------------+
|food_groups_tags|
+----------------+
+----------------+



En nous basant sur un format ne contenant aucuns chiffres, aucune valeur aberrante n'a été détectée pour notre colonne food_groups_tags.

_ _ _

# Vérification et exportation du DataFrame

## DataFrame finalisé

Afin d'obtenir une vue synthétique des transformations effectuées sur notre Dataframe initial, nous présenterons succinctement les principales caractéristiques de notre jeu de données avant, puis après modification.

Nombre de colonnes avant traitement : 206

In [77]:
# Obtenir le nombre de colonnes actuel

nb_colonnes = len(df_cleaned_10.columns)

print(f"\nNombre de colonnes actuel : {nb_colonnes}\n")


Nombre de colonnes actuel : 12



Nombre de lignes avant traitement : 3083356

In [78]:
# Affichons le nombre de lignes de notre DataFrame final
print(f"Nombre de lignes actuel : {df_cleaned_10.count()}")

Nombre de lignes actuel : 3732


Affichons les 20 premières lignes de notre DataFrame :

In [79]:
# Nous affichons notre DataFrame finalisé
df_cleaned_10.show(20,truncate=False)

+----------------------------------------------------+----------------+---------------------------------------------+----------------+------------------+------------------+-----------+----------+-------------+---------+-----------+--------------+
|product_name                                        |countries_tags  |food_groups_tags                             |energy-kcal_100g|saturated-fat_100g|carbohydrates_100g|sugars_100g|fiber_100g|proteins_100g|salt_100g|sodium_100g|vitamin-c_100g|
+----------------------------------------------------+----------------+---------------------------------------------+----------------+------------------+------------------+-----------+----------+-------------+---------+-----------+--------------+
|Great Northern Beans                                |en:united-states|en:cereals-and-potatoes,en:legumes           |69.0            |0.0               |12.31             |1.54       |4.6       |5.38         |0.7875   |0.315      |0.0           |
|100% Natura

Ainsi, comme nous pouvons le constater, l'opération de qualité de données effectuée sur notre jeu de données initial a considérablement réduit cette masse de données brute afin de ne garder que celles utilent pour nos besoins futurs.